<a href="https://colab.research.google.com/github/JimKing100/NFL-Live/blob/master/data-science/model-rnn/Predictions_Defense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Imports
import numpy as np
import pandas as pd

from sklearn import preprocessing

from pandas import DataFrame
from pandas import Series
from pandas import concat

from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from math import sqrt
from matplotlib import pyplot

import numpy

Using TensorFlow backend.


In [0]:
# Import data
defense_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/NFL-Live/master/data-science/data/actuals/actuals_defense.csv')
player_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/NFL-Live/master/data-science/data/raw/players.csv')
bye_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/NFL-Live/master/data-science/data/raw/bye.csv')

In [0]:
# The dataframe of actual defense points for each game from 2000-2019
defense_df.head()

,player,first,last,name,position1,2000-game1,2000-game2,2000-game3,2000-game4,2000-game5,2000-game6,2000-game7,2000-game8,2000-game9,2000-game10,2000-game11,2000-game12,2000-game13,2000-game14,2000-game15,2000-game16,2001-game1,2001-game2,2001-game3,2001-game4,2001-game5,2001-game6,2001-game7,2001-game8,2001-game9,2001-game10,2001-game11,2001-game12,2001-game13,2001-game14,2001-game15,2001-game16,2002-game1,2002-game2,2002-game3,...,2017-game9,2017-game10,2017-game11,2017-game12,2017-game13,2017-game14,2017-game15,2017-game16,2018-game1,2018-game2,2018-game3,2018-game4,2018-game5,2018-game6,2018-game7,2018-game8,2018-game9,2018-game10,2018-game11,2018-game12,2018-game13,2018-game14,2018-game15,2018-game16,2019-game1,2019-game2,2019-game3,2019-game4,2019-game5,2019-game6,2019-game7,2019-game8,2019-game9,2019-game10,2019-game11,2019-game12,2019-game13,2019-game14,2019-game15,2019-game16
0,SF,San Francisco,49ers,San Francisco 49ers,DF,-1.0,1.0,2.0,9.0,2.0,4.0,1.0,2.0,3.0,3.0,8.0,16.0,19.0,4.0,8.0,1.0,5.0,6.0,5.0,8.0,3.0,15.0,6.0,3.0,0.0,17.0,14.0,8.0,20.0,9.0,3.0,17.0,12,6,12,...,4,5,2,3,6,3,12,10,7.0,4.0,-1.0,8.0,0.0,2.0,-1.0,9.0,11.0,1.0,1.0,0.0,3.0,3.0,8.0,-5.0,24,7,6,14,9,9,16,3,19,14,10,3,-3,1,7,1
1,CHI,Chicago,Bears,Chicago Bears,DF,2.0,-1.0,9.0,7.0,8.0,9.0,4.0,8.0,12.0,3.0,19.0,3.0,0.0,3.0,8.0,15.0,5.0,10.0,26.0,14.0,8.0,14.0,11.0,5.0,10.0,9.0,10.0,5.0,13.0,11.0,26.0,15.0,7,11,6,...,3,5,4,5,10,6,11,8,14.0,17.0,13.0,13.0,5.0,2.0,4.0,27.0,12.0,14.0,13.0,4.0,17.0,8.0,6.0,7.0,8,5,21,14,4,-2,4,4,7,5,4,4,2,1,1,8
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,3.0,9.0,-1.0,5.0,12.0,2.0,8.0,8.0,1.0,2.0,4.0,3.0,8.0,1.0,4.0,11.0,5.0,23.0,1.0,7.0,5.0,2.0,10.0,2.0,5.0,9.0,9.0,13.0,1.0,8.0,20.0,7.0,0,1,0,...,6,10,5,3,1,3,7,9,12.0,12.0,0.0,0.0,22.0,-1.0,1.0,19.0,-5.0,4.0,-3.0,4.0,2.0,10.0,2.0,10.0,6,-1,5,2,1,9,2,0,-2,8,6,8,5,1,1,12
3,BUF,Buffalo,Bills,Buffalo Bills,DF,9.0,8.0,2.0,3.0,3.0,10.0,1.0,15.0,10.0,20.0,12.0,6.0,3.0,-1.0,4.0,13.0,2.0,10.0,2.0,-3.0,11.0,1.0,11.0,13.0,0.0,3.0,1.0,10.0,12.0,10.0,7.0,2.0,2,14,3,...,-3,-4,7,5,4,9,7,4,-1.0,1.0,14.0,6.0,11.0,15.0,-3.0,2.0,0.0,10.0,7.0,5.0,2.0,3.0,7.0,13.0,7,6,11,3,10,11,4,7,4,9,10,11,3,17,2,7
4,DEN,Denver,Broncos,Denver Broncos,DF,11.0,13.0,18.0,13.0,10.0,13.0,22.0,3.0,6.0,18.0,8.0,12.0,9.0,14.0,10.0,13.0,5.0,10.0,2.0,16.0,9.0,7.0,18.0,-1.0,12.0,10.0,6.0,2.0,12.0,11.0,10.0,2.0,8,7,12,...,-2,4,4,11,11,4,4,12,12.0,3.0,6.0,1.0,4.0,7.0,31.0,3.0,4.0,7.0,11.0,13.0,7.0,7.0,1.0,6.0,0,1,0,7,11,17,0,7,2,7,3,7,15,5,5,5


In [0]:
# The dataframe of all 2019 players
player_df.head()

,player,first,last,name,position1,start
0,SF,San Francisco,49ers,San Francisco 49ers,DF,2000
1,CHI,Chicago,Bears,Chicago Bears,DF,2000
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,2000
3,BUF,Buffalo,Bills,Buffalo Bills,DF,2000
4,DEN,Denver,Broncos,Denver Broncos,DF,2000


In [0]:
# The dataframe of bye weeks for 2019 teams
bye_df.head()

,player,first,last,name,position1,bye-week
0,SF,San Francisco,49ers,San Francisco 49ers,DF,4
1,CHI,Chicago,Bears,Chicago Bears,DF,6
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,9
3,BUF,Buffalo,Bills,Buffalo Bills,DF,6
4,DEN,Denver,Broncos,Denver Broncos,DF,10


In [0]:
# Frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

In [0]:
# Scale train and test data to [-1, 1]
def scale(train, test):
    # Fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    # Transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # Transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled

In [0]:
# Inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = numpy.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]

In [0]:
# Fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
      model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
      model.reset_states()
    return model

In [0]:
# Make a one-step forecast
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]

In [0]:
# Create the time series for a team
def make_series(p):
    column_names = ['game', 'points']
    df = pd.DataFrame(columns = column_names)

    for i in range (1, 321):
        game = i
        col = i + 4
        points = defense_df.iloc[:, col].loc[(defense_df['player']==p)].iloc[0]
        df = df.append({'game': game, 'points': points}, ignore_index=True)

    s = pd.Series(df['points'].values, index=df['game'])
    return s

In [0]:
def lstm_pred(team, np):
    series = make_series(team)
    X = series.values
    supervised = timeseries_to_supervised(X, 1)
    supervised_values = supervised.values

    # Split data into train and test-sets
    train, test = supervised_values[0:-np], supervised_values[-np:]

    # Transform the scale of the data
    scaler, train_scaled, test_scaled = scale(train, test)

    # Fit the model
    lstm_model = fit_lstm(train_scaled, 1, 100, 1)

    # Forecast the entire training dataset to build up state for forecasting
    train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
    lstm_model.predict(train_reshaped, batch_size=1)

    # Walk-forward validation on the test data
    yhat_sum = 0
    for i in range(len(test_scaled)):
        # Make one-step forecast
        X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
        yhat = forecast_lstm(lstm_model, 1, X)
        # Invert scaling
        yhat = invert_scale(scaler, X, yhat)
        # Sum the weekly forecasts
        yhat_sum = yhat_sum + yhat

    return yhat_sum

In [0]:
# Add a row to the final_df dataframe
# Each row represents the predicted points for each team
def add_row(df, p, f, l, n, pos, wc, wp, wa, cur, pred, act):

    df = df.append({'player': p,
                    'first': f,
                    'last': l,
                    'name': n,
                    'position': pos,
                    wc: cur,
                    wp: pred,
                    wa: act
                    }, ignore_index=True)
    
    return df

In [0]:
# The main code for iterating through the player(defense) list, calculating the points and adding the rows
# to the final_df dataframe.
def main(w):

    # Create the column names
    week_cname = 'week' + str(w) + '-cur'
    week_pname = 'week' + str(w) + '-pred'
    week_aname = 'week' + str(w) + '-act'

    column_names = ['player',
                    'first',
                    'last',
                    'name',
                    'position',
                    week_cname, 
                    week_pname,
                    week_aname
                     ]

    d_df = player_df.loc[player_df['position1'] == 'DF']
    player_list = d_df['player'].tolist()

    # The row is the current row of the dataframe (0)
    # The column is the column of the dataframe (309 = 2019-game1)
    row = 0
    col = 308 + w

    week_df = pd.DataFrame(columns = column_names)

    for player in player_list:

        first = player_df['first'].loc[(player_df['player']==player)].iloc[0]
        last = player_df['last'].loc[(player_df['player']==player)].iloc[0]
        name = player_df['name'].loc[(player_df['player']==player)].iloc[0]
        position1 = player_df['position1'].loc[(player_df['player']==player)].iloc[0]

        bye_week = bye_df['bye-week'].loc[(bye_df['player']==player)].iloc[0]

        # Handle bye week
        if w > bye_week:
            new_col = col - 1
        else:
            new_col = col

        # Handle Houston (started in 2002)
        if player == 'HOU':
            train_data = defense_df.iloc[row, 37:new_col]
        else:
            train_data = defense_df.iloc[row, 5:new_col]

        # Calculate current points
        if w == 1:
            current = defense_df.iloc[row, 293:309]
        else:
            current = defense_df.iloc[row, 309:new_col]
        cur_points = current.sum()

        # Calculate actual points
        actuals = defense_df.iloc[row, new_col:325]
        act_points = actuals.sum()
        row = row + 1
        
        # Forecast
        if w > bye_week:
            n_periods = 17 - w + 1
        else:
            n_periods = 17 - w
 
        # Calculate predicted points
        print(player)
        pred_points = lstm_pred(player, n_periods)

        week_df = add_row(week_df, player, first, last, name, position1, week_cname, 
                          week_pname, week_aname, cur_points, pred_points, act_points)
    return week_df

In [0]:
# Run for seasons 1 through 17
for i in range(6, 8):
    week = i
    final_df = main(week)

    week_pred = 'week' + str(week) + '-pred'
    week_act = 'week' + str(week) + '-act'
    week_diff = 'week' + str(week) + '-diff'
    week_pct = 'week' + str(week) + '-pct'

    final_df[week_diff] = final_df[week_pred] - final_df[week_act]
    final_df[week_pct] = final_df[week_diff] / final_df[week_pred]
    final_df[week_pred] = final_df[week_pred].astype(int)

    file_name = '/content/week' + str(week) + '-pred-defense.csv'
    final_df.to_csv(file_name, index=False)

SF
CHI
CIN
BUF
DEN
CLE
TB
ARI
LAC
KC
IND
DAL
MIA
PHI
ATL
NYG
JAC
NYJ
DET
GB
CAR
NE
OAK
LA
BAL
WAS
NO
SEA
PIT
HOU
TEN
MIN
SF
CHI
CIN
BUF
DEN
CLE
TB
ARI
LAC
KC
IND
DAL
MIA
PHI
ATL
NYG
JAC
NYJ
DET
GB
CAR
NE
OAK
LA
BAL
WAS
NO
SEA
PIT
HOU
TEN
MIN
